In [1]:
import pandas as pd

# Importing Gensim
import gensim
from gensim import corpora

In [10]:
df = pd.read_csv('all_corpus.csv')

corpus = list(df['Text'])

In [11]:
doc_clean = []
for i in range(len(corpus)):
    doc_clean.append(corpus[i].split())

print(doc_clean[0])

['thank', 'welcome', 'amd', 'first', 'quarter', 'conference', 'call', 'opportunity', 'review', 'copy', 'earning', 'release', 'cfo', 'commentary', 'accompany', 'slide', 'review', 'document', 'find', 'amd', 'website', 'iramdcom']


In [12]:
# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [13]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50)

In [14]:
print(ldamodel.print_topics(num_topics=3, num_words=3))

[(0, '0.016*"think" + 0.015*"go" + 0.014*"see"'), (1, '0.029*"revenue" + 0.028*"quarter" + 0.023*"million"'), (2, '0.008*"really" + 0.007*"be" + 0.007*"cloud"')]


#### Prep data for pyLDAvis

In [15]:
# sklearn
from sklearn import datasets
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# logging (set to INFO)
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [91]:
from sklearn.feature_extraction import text 

additional_sw = ['thank','caller','okay','join','operator','stacy','pron','question','good','great',
                'think','time','wea','look','say','yes','really','thing','ita','talk','number','right',
                 'overall','thata','probably','dont', 'second','followon','chris','john','ambrish',
                 'joe','thanks','amit','ross','harlan','really','yes','inside','weve','thats','change',
                'turn','let','dave','hand','mark','ill','eyal','comment','bob','laura','brian','devinder',
                 'answer','wrap','forwardlooke','materially','contain','cause','today','discussion',
                 'involve','rafael','review','followup','ask','outlook','order','additional','limit','response',
                'ray','refer','appreciate','evening','kevin','february','lisa','afternoon','welcome','want','actually']

stop_words = text.ENGLISH_STOP_WORDS.union(additional_sw)

tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = stop_words,
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [92]:
# generating an alternative model using TF-IDF:
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(corpus)
#print(dtm_tfidf.shape)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1799: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. <class 'numpy.int64'> 'dtype' will be converted to np.float64.
  UserWarning)


In [105]:
# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_components=6, random_state=42)
lda_tfidf.fit(dtm_tfidf)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


LatentDirichletAllocation(n_components=6, random_state=42)

### Visualizing with pyLDAvis

In [120]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer, mds='mmds')
#vis = pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer, mds='mmds')
#pyLDAvis.save_html(vis, 'view_pyLDAvis.html')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.017506  0.105169       1        1  27.494134
2     -0.055739 -0.106192       2        1  26.908868
0     -0.231030 -0.076399       3        1  18.040945
1     -0.059786  0.174321       4        1  13.620506
4      0.118610 -0.154292       5        1   8.783889
3      0.210438  0.057394       6        1   5.151658, topic_info=     Category        Freq           Term       Total  loglift  logprob
134   Default   66.000000           cash   66.000000  30.0000  30.0000
694   Default   98.000000        million   98.000000  29.0000  29.0000
127   Default   40.000000        capital   40.000000  28.0000  28.0000
442   Default   44.000000           flow   44.000000  27.0000  27.0000
653   Default   27.000000     management   27.000000  26.0000  26.0000
621   Default   50.000000           like   50.000000  25.0000  25.0000
457   Default   36.000000           free   36.000000  24.0000  24.0000
549   Default   29.000000     industrial   29.000000  23.0000  23.0000
74    Default   29.000000     automotive   29.000000  22.0000  22.0000
385   Default   30.000000       ethernet   30.000000  21.0000  21.0000
436   Default   35.000000      financial   35.000000  20.0000  20.0000
94    Default   66.000000        billion   66.000000  19.0000  19.0000
445   Default   24.000000         follow   24.000000  18.0000  18.0000
659   Default   77.000000         market   77.000000  17.0000  17.0000
885   Default  144.000000        quarter  144.000000  16.0000  16.0000
865   Default   19.000000  profitability   19.000000  15.0000  15.0000
957   Default  117.000000        revenue  117.000000  14.0000  14.0000
1125  Default   29.000000            tax   29.000000  13.0000  13.0000
49    Default   14.000000        analyst   14.000000  12.0000  12.0000
413   Default   33.000000        expense   33.000000  11.0000  11.0000
466   Default   33.000000           game   33.000000  10.0000  10.0000
1234  Default   19.000000        website   19.000000   9.0000   9.0000
874   Default   36.000000        provide   36.000000   8.0000   8.0000
552   Default   25.000000     infiniband   25.000000   7.0000   7.0000
658   Default   53.000000         margin   53.000000   6.0000   6.0000
939   Default   13.000000         replay   13.000000   5.0000   5.0000
992   Default   16.000000         sector   16.000000   4.0000   4.0000
378   Default   20.000000      equipment   20.000000   3.0000   3.0000
737   Default   33.000000        nongaap   33.000000   2.0000   2.0000
763   Default   35.000000        operate   35.000000   1.0000   1.0000
...       ...         ...            ...         ...      ...      ...
283    Topic6    2.763635     department    3.538032   2.7188  -5.5275
665    Topic6    6.962936           matt    8.956880   2.7140  -4.6034
873    Topic6    4.269758          proud    5.712960   2.6747  -5.0925
122    Topic6    3.685370       calendar    5.046750   2.6515  -5.2397
626    Topic6    5.114661         listen    7.182479   2.6263  -4.9119
204    Topic6    4.816963       conclude    7.008920   2.5908  -4.9719
279    Topic6    3.522148           dell    5.212495   2.5739  -5.2850
103    Topic6    4.183893          break    6.259529   2.5630  -5.1128
993    Topic6    2.406414         secure    3.666371   2.5448  -5.6659
865    Topic6   12.612733  profitability   19.248154   2.5431  -4.0093
263    Topic6    6.135427          david    9.407840   2.5384  -4.7300
939    Topic6    8.824682         replay   13.967013   2.5067  -4.3665
653    Topic6   16.870009     management   27.198923   2.4882  -3.7185
169    Topic6    6.464885          color   10.445197   2.4861  -4.6777
974    Topic6    2.554032         safety    4.129994   2.4852  -5.6064
1107   Topic6    2.926646      summarize    4.960965   2.4381  -5.4702
512    Topic6    2.323843      highlevel    4.326925   2.3442  -5.7008
1193   Topic6    1.878075         united    3.

In [50]:
from sklearn.decomposition import NMF

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [121]:
# Use NMF to look for 15 topics
n_topics = 10
model = NMF(n_components=n_topics)
model.fit(dtm_tfidf)

# Print the top 10 words
n_words = 15
feature_names = tfidf_vectorizer.get_feature_names()

topic_list = []
for topic_idx, topic in enumerate(model.components_):
    top_n = [feature_names[i]
             for i in topic.argsort()
             [-n_words:]][::-1]
    top_features = ' '.join(top_n)
    topic_list.append(f"topic_{'_'.join(top_n[:3])}") 

    print(f"Topic {topic_idx}: {top_features}")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Topic 0: revenue margin gross year profit ago yearoveryear billion segment low increase operate drive point high
Topic 1: million nongaap expense compare operate income charge approximately net share prior debt gaap diluted operating
Topic 2: cash flow free billion return dividend owner capital shareholder repurchase value trail operation period stock
Topic 3: cloud center platform datum new game data gpu performance gpus inference business compute world architecture
Topic 4: inventory year half little expect bit level product customer channel demand kind know start come
Topic 5: quarter fourth revenue decline result financial end increase strong total represent demand provide color start
Topic 6: website financial release statement earning risk available replay result web conference chief officer current differ
Topic 7: market growth continue industrial automotive grow business focus embed analog product investment opportunity provide year
Topic 8: ethernet infiniband gigabit switch r

In [ ]:
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_tfidf, dtm_tfidf, dictionary)

In [ ]:
vis_data = pyLDAvis.gensim.prepare(lda_tfidf, dtm_tfidf, dictionary)
pyLDAvis.save_html(vis_data, 'output_filename.html')